In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/short-term-load-forecasting/sample_submission.csv
/kaggle/input/short-term-load-forecasting/train.csv
/kaggle/input/short-term-load-forecasting/metadata.csv
/kaggle/input/short-term-load-forecasting/test.csv


In [2]:
!pip install sktime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.5/35.5 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.7/145.7 kB 7.1 MB/s eta 0:00:00


In [3]:
timestamp_column = 'timestamp'
test_df = pd.read_csv('/kaggle/input/short-term-load-forecasting/test.csv', parse_dates=[timestamp_column])
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 676800 entries, 0 to 676799
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   row_id         676800 non-null  int64         
 1   building_id    676800 non-null  object        
 2   window_id      676800 non-null  int64         
 3   timestamp      676800 non-null  datetime64[ns]
 4   meter_reading  592200 non-null  float64       
 5   role           676800 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 31.0+ MB


In [4]:
from tqdm import tqdm
from sktime.forecasting.ttm import TinyTimeMixerForecaster

CONTEXT_LENGTH = 168
PREDICTION_LENGTH = 24

In [5]:
# from sktime.forecasting.ttm import TinyTimeMixerForecaster
# from sktime.datasets import load_airline
# y = load_airline()
# forecaster = TinyTimeMixerForecaster() 
# # performs zero-shot forecasting, as default config (unchanged) is used
# forecaster.fit(y, fh=[1, 2, 3]) 
# y_pred = forecaster.predict() 

Forecast one window with TTM zero-shot

See. https://www.sktime.net/en/latest/api_reference/auto_generated/sktime.forecasting.ttm.TinyTimeMixerForecaster.html#

In [6]:
test_df1 = test_df[test_df.window_id == 1]
test_df1_in = test_df1[test_df1.role == 'input']
test_df1_in = test_df1_in[['timestamp', 'meter_reading']]
test_df1_in.set_index('timestamp', inplace=True)
test_df1_in

,meter_reading
timestamp,
2019-07-09 00:00:00,0.6062
2019-07-09 01:00:00,0.8477
2019-07-09 02:00:00,0.3375
2019-07-09 03:00:00,0.5490
2019-07-09 04:00:00,0.3079
...,...
2019-07-15 19:00:00,0.5955
2019-07-15 20:00:00,1.1690
2019-07-15 21:00:00,1.6299


In [7]:
ttm = TinyTimeMixerForecaster(fit_strategy = 'zero-shot') 
#ttm = TinyTimeMixerForecaster(fit_strategy = 'minimal') 
# performs zero-shot forecasting, as default config (unchanged) is used
fh_24 = [i+1 for i in range(24)]
ttm.fit(test_df1_in, fh=fh_24) 
y_pred = ttm.predict() 
y_pred

2025-08-12 20:26:32.482238: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755030392.721869      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755030392.789927      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/3.24M [00:00<?, ?B/s]

,meter_reading
timestamp,
2019-07-16 00:00:00,0.549097
2019-07-16 01:00:00,0.601013
2019-07-16 02:00:00,0.617490
2019-07-16 03:00:00,0.622912
2019-07-16 04:00:00,0.611179
2019-07-16 05:00:00,0.617065
2019-07-16 06:00:00,0.666165
2019-07-16 07:00:00,0.717604
2019-07-16 08:00:00,0.692015


### forecast each window iteratively

In [8]:

# forecasted_values = []
# for w_id in tqdm(test_df.window_id.unique()):
#     ## filter the window

#     win = test_df[test_df.window_id == w_id]
#     win = win[win.role == 'input']
#     win = win[['timestamp', 'meter_reading']]
#     win.set_index('timestamp', inplace=True)
#     win    

#     #ttm = TinyTimeMixerForecaster(fit_strategy = 'zero-shot') 
#     fh_24 = [i+1 for i in range(PREDICTION_LENGTH)]
#     ttm.fit(win, fh=fh_24) 
#     y_pred = ttm.predict()

#     forecasted_values.extend(y_pred.meter_reading.to_list())

### forecast with multi-index -- forecast all windows together

In [9]:
# from sktime.forecasting.ttm import TinyTimeMixerForecaster
# from sktime.datasets import load_tecator
# # load multi-index dataset
# y = load_tecator(
#     return_type="pd-multiindex",
#     return_X_y=False
# )
# y.drop(['class_val'], axis=1, inplace=True)
# # global forecasting on multi-index dataset
# forecaster = TinyTimeMixerForecaster(
#     model_path=None,
#     fit_strategy="full",
#     config={
#             "context_length": 8,
#             "prediction_length": 2
#     },
#     training_args={
#         "num_train_epochs": 1,
#         "output_dir": "test_output",
#         "per_device_train_batch_size": 32,
#     },
# ) 
# # model initialized with random weights due to None model_path
# # and trained with the full strategy.
# forecaster.fit(y, fh=[1, 2, 3]) 
# y_pred = forecaster.predict() 

In [10]:
print(test_df.shape)
test_df1 = test_df.copy()
test_df1_in = test_df1[test_df1.role == 'input']

## Note: multi-index forecasting expects same index for all series
#test_df1_in['seq'] = test_df1_in.timestamp.dt.hour
test_df1_in['seq'] = [i+1 for i in range(CONTEXT_LENGTH)] * test_df.window_id.max()

test_df1_in = test_df1_in[['window_id', 'seq', 'meter_reading']]
test_df1_in.set_index(['window_id', 'seq'], inplace=True)
test_df1_in

(676800, 6)


/tmp/ipykernel_13/4026684502.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df1_in['seq'] = [i+1 for i in range(CONTEXT_LENGTH)] * test_df.window_id.max()


meter_reading
window_id seq               
1         1           0.6062
          2           0.8477
          3           0.3375
          4           0.5490
          5           0.3079
...                      ...
3525      164         1.4808
          165         0.6253
          166         0.7634
          167         1.1569
          168         1.0869

[592200 rows x 1 columns]

In [11]:
ttm = TinyTimeMixerForecaster() 
fh_24 = [i+1 for i in range(PREDICTION_LENGTH)]
ttm.fit(test_df1_in, fh=fh_24) 
y_pred = ttm.predict() 

y_pred

meter_reading
window_id seq               
1         169       0.549097
          170       0.601013
          171       0.617490
          172       0.622912
          173       0.611179
...                      ...
3525      188       1.157741
          189       1.092202
          190       1.069357
          191       1.078920
          192       1.097518

[84600 rows x 1 columns]

In [12]:
y_pred.describe()

,meter_reading
count,84600.000000
mean,0.910812
std,0.718788
min,-0.000389
25%,0.493459
50%,0.758287
75%,1.097479
max,8.405735


In [13]:
submission = pd.read_csv('/kaggle/input/short-term-load-forecasting/sample_submission.csv')
print(submission.shape)
submission

(84600, 2)


,row_id,meter_reading
0,169,0
1,170,0
2,171,0
3,172,0
4,173,0
...,...,...
84595,676796,0
84596,676797,0
84597,676798,0
84598,676799,0


In [14]:
# y_pred.meter_reading.to_list()

In [15]:
submission.meter_reading = y_pred.meter_reading.to_list()
submission.to_csv("submission.csv", index=False)

In [16]:
submission

,row_id,meter_reading
0,169,0.549097
1,170,0.601013
2,171,0.617490
3,172,0.622912
4,173,0.611179
...,...,...
84595,676796,1.157741
84596,676797,1.092202
84597,676798,1.069357
84598,676799,1.078920
